## Libraries

In [ ]:
import os
from tqdm import tqdm
import glog
import pickle

import torch
print('Cuda available:', torch.cuda.is_available())
cuda_id = torch.cuda.current_device()
print('Cuda_id: ', cuda_id)
print(torch.cuda.get_device_name(cuda_id))

Cuda available: True
Cuda_id:  0
NVIDIA GeForce RTX 4060 Laptop GPU


## JL Auxiliary Functions

In [2]:
import os
import pickle

# Function to save data with a flag to prevent overwriting
def save_data(file_path, data):
    if os.path.exists(file_path):
        print(f"The file {file_path} exist and will returned")
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    
    else: 
        with open(file_path, 'wb') as f:
            pickle.dump(data, f)
        print(f"Data has been saved to {file_path}.")
        return data

## Read Candidates from Module 1

In [3]:
args_dataset = 'conan' # default='sample', choices=['reddit', 'gab', 'conan']
args_kpq = 300

In [4]:
from utils_JL import read_candidates
# Load generated candidates from VAE
candidates = read_candidates('./data/' + args_dataset + '_candidates.txt')  
candidates[:5]

2024-12-01 16:01:16.803440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I1201 16:01:18.042452 2093 utils.py:148] Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
I1201 16:01:18.043147 2093 utils.py:161] NumExpr defaulting to 16 threads.


["show hate of the entire hate and are isn't allowed to know? i this this.",
 "if they say ban war out british crimes. i cannot extend i am judge what here i just need to an inner news am really be good 'fundmentalism' and first been true and islam will not confuse intolerant, track neighbors.",
 "muslims as where? are convicted more our means but well the threat that is what it it is even say islam does not a hate to automatically dangerous, help their justified'.",
 "how aren't seems that the violent of was actions of their rest of the western world.",
 "shall do you see that a news for people with the government, of some evil'?"]

In [5]:
from utils_JL import initialize_train_test_dataset
train_x_text, train_y_text, test_x_text, test_y_text = initialize_train_test_dataset(args_dataset)

train_x_text = save_data('backup/train_x_text.pkl', train_x_text)
train_y_text = save_data('backup/train_y_text.pkl', train_y_text)

test_x_text = save_data('backup/test_x_text.pkl', test_x_text)
test_y_text = save_data('backup/test_y_text.pkl', test_y_text)

print(train_x_text[0])
print(train_y_text[0])

The file backup/train_x_text.pkl exist and will returned
The file backup/train_y_text.pkl exist and will returned
The file backup/test_x_text.pkl exist and will returned
The file backup/test_y_text.pkl exist and will returned
i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.
["Using words that insult one group while defending another group doesn't come across as helpful.", 'You can make the same point more effectively without the use of hateful terminology.', 'Use of the r-word is unacceptable in our discourse as it demeans and insults people with mental disabilities.']


In [7]:
from utils_JL import convert_to_contexts_responses

contexts_train, responses_train = convert_to_contexts_responses(train_x_text, train_y_text)
contexts_train = save_data('backup/contexts_train.pkl', contexts_train)
responses_train = save_data('backup/responses_train.pkl', responses_train)

print(contexts_train[0])
print(responses_train[0])

The file backup/contexts_train.pkl exist and will returned
The file backup/responses_train.pkl exist and will returned
i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.
You can make the same point more effectively without the use of hateful terminology.


## Module 2: Candidates Pruning by Grammaticality

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

print('Cuda available:', torch.cuda.is_available())
cuda_id = torch.cuda.current_device()
print('Cuda_id: ', cuda_id)
print(torch.cuda.get_device_name(cuda_id))


Cuda available: True
Cuda_id:  0
NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
from language_quality import extract_good_candidates_by_LQ
candidates = extract_good_candidates_by_LQ(candidates, LQ_thres=0.52, num_of_generation=30000)
# candidates = save_data('backup/extracted_good_candidates_by_LQ.pkl', candidates)

W1201 15:27:36.573204 361009 warnings.py:109] /home/jaimelicea/miniconda3/envs/gps/lib/python3.9/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this expe

In [9]:
candidates = save_data('backup/extracted_good_candidates_by_LQ.pkl', None)
candidates[:5]

The file backup/extracted_good_candidates_by_LQ.pkl exist and will returned


['I think the government has a very good understanding of who should be deported and members of the public should leave this type of decisions to them?',
 'how am you say that?',
 'Small groups of people peacefully gather to pray, but because they do not look like you, you are outraged? Why am I not surprised?',
 'The proportion of Muslims unemployed is no more than that of non-Muslims (add a link to UK employment statistics).',
 'provide positive']

In [10]:
from utility.response_selection import keyword_based
from utility.response_selection import vector_based_JL
from utility.response_selection import method
from utils_JL import to_method_object

from tqdm import tqdm

In [11]:
args_kpq = 300

method = to_method_object('TF_IDF')
method.train(contexts_train, responses_train)
 # kpq: Top k candidates per query, for better computation.
good_candidates_index = method.sort_responses(test_x_text, candidates, min(args_kpq, len(candidates))) 
good_candidates = [[candidates[y] for y in x] for x in good_candidates_index]
good_candidates[1][:5]

good_candidates = save_data('backup/good_candidates.pkl', good_candidates)


Data has been saved to backup/good_candidates.pkl.


## Module 3: Response Selection (env: py7_tf1_v03)

In [1]:
from utility.response_selection import keyword_based
from utility.response_selection import method

from tqdm import tqdm
import pandas as pd
import numpy as np


In [2]:
args_dataset = 'conan' # default='sample', choices=['reddit', 'gab', 'conan']
args_kpq = 300

In [3]:
import os
import pickle

# Function to save data with a flag to prevent overwriting
def save_data(file_path, data):
    if os.path.exists(file_path):
        print(f"The file {file_path} exist and will returned")
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    
    else: 
        with open(file_path, 'wb') as f:
            pickle.dump(data, f)
        print(f"Data has been saved to {file_path}.")
        return data

In [5]:
contexts_train = save_data('backup/contexts_train.pkl', None)
responses_train = save_data('backup/responses_train.pkl', None)

train_x_text = save_data('backup/train_x_text.pkl', None)
train_y_text = save_data('backup/train_y_text.pkl', None)
test_x_text = save_data('backup/test_x_text.pkl', None)
test_y_text = save_data('backup/test_y_text.pkl', None)

candidates = save_data('backup/extracted_good_candidates_by_LQ.pkl', None)
good_candidates = save_data('backup/good_candidates.pkl', None)

The file backup/contexts_train.pkl exist and will returned
The file backup/responses_train.pkl exist and will returned
The file backup/train_x_text.pkl exist and will returned
The file backup/train_y_text.pkl exist and will returned
The file backup/test_x_text.pkl exist and will returned
The file backup/test_y_text.pkl exist and will returned
The file backup/extracted_good_candidates_by_LQ.pkl exist and will returned
The file backup/good_candidates.pkl exist and will returned


In [17]:
from utility.response_selection import vector_based_JL as vector_based

method = vector_based.VectorSimilarityMethod(encoder=vector_based.TfHubEncoder("https://tfhub.dev/google/universal-sentence-encoder/2"))
# method = vector_based.VectorSimilarityMethod(encoder=vector_based.SentenceTransformerEncoder('sentence-transformers/use-cmlm-multilingual'))

method.train(contexts_train, responses_train)

output = []
for i, test_i in enumerate(tqdm(test_x_text)):
    # print(i)
    predictions = method.rank_responses([test_i], good_candidates[i])
    output.append(good_candidates[i][predictions.item()])
print(output)

df_output = pd.DataFrame({'x_text': test_x_text, 'output': output})
df_output = save_data('backup/df_output_SIM_EN.pkl', df_output)

/home/jaimelicea/miniconda3/envs/py7_tf1_v03/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1201 16:12:28.786809 2668 module_wrapper.py:139] From /home/jaimelicea/Documentos/Git_Projects/GPS/utility/response_selection/vector_based_JL.py:58: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2024-12-01 16:12:28.789915: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 16:12:28.808785: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2803195000 Hz
2024-12-01 16:12:28.811472: I tensorflow/

['Wow, pipe down. Why so angry?', 'Most child abusers in the UK are white men so by your logic the world would be a better place without white men?', "The world is not lost. We all face challenges, but is someone's culture is different from ours, it does not mean that they are evil and should be banned.", 'why you you think that what is just an entire them? as anti-semitism? a offensive large muslim? we do not deal with muslims and people.', 'You cannot be racist towards an ideology. You can, however, be discriminatory and intolerant towards the people that follow it.', 'Do you know that anti-Islamism is actually a form of anti-Semitism? It is equally wrong to hate Jews and Muslims.', "How many unemployed 'lazy' Muslims do you know? There are no more work-shy muslims than there are people of any other faith.", "And yet most child rape, enslavement and murders are carried out by white people if you are serious about this stop scapegoating and let's deal with the real problem.", 'Can you

In [18]:
from utility.response_selection import vector_based_JL as vector_based

method = vector_based.VectorMappingMethod(encoder=vector_based.TfHubEncoder("https://tfhub.dev/google/universal-sentence-encoder/2"))
# method = vector_based.VectorMappingMethod(encoder=vector_based.SentenceTransformerEncoder('sentence-transformers/use-cmlm-multilingual'))

method.train(contexts_train, responses_train)

output = []
for i, test_i in enumerate(tqdm(test_x_text)):
    # print(i)
    predictions = method.rank_responses([test_i], good_candidates[i])
    output.append(good_candidates[i][predictions.item()])
print(output)

df_output = pd.DataFrame({'x_text': test_x_text, 'output': output})
df_output = save_data('backup/df_output_MAP_EN.pkl', df_output)

I1201 16:17:02.543278 2668 vector_based_JL.py:60] Loading https://tfhub.dev/google/universal-sentence-encoder/2 model from tensorflow hub
I1201 16:17:03.307139 2668 saver.py:1503] Saver not created because there are no variables in the graph to restore
I1201 16:17:03.357636 2668 vector_based_JL.py:65] Initializing graph.
I1201 16:17:07.542687 2668 vector_based_JL.py:117] Training on 11691 contexts and responses.
I1201 16:17:07.543504 2668 vector_based_JL.py:130] Encoding the train set.
100%|██████████| 117/117 [00:03<00:00, 35.59it/s]
I1201 16:17:10.855472 2668 vector_based_JL.py:119] Created a training set of size 9352, and a dev set of size 2339.
W1201 16:17:10.880072 2668 deprecation.py:323] From /home/jaimelicea/Documentos/Git_Projects/GPS/utility/response_selection/vector_based_JL.py:154: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to 

['wow, tea.', 'Says who?', "The world is not lost. We all face challenges, but is someone's culture is different from ours, it does not mean that they are evil and should be banned.", 'Why? What is wrong about learning about a world religion that many of their peers practice?', 'Everyone is a potential everything, what would you like to be?', 'Do you know that anti-Islamism is actually a form of anti-Semitism? It is equally wrong to hate Jews and Muslims.', 'How old are you? The first Mosque in the UK was build in 1889!', "Well studies show that people from ethnic minorities have to send 74% more applications to achieve equal success to White people. There's your cause of unemployment.", 'Are you a misogynistic man trying to use objectification of women to justify your hate?', 'You obviously do not understand what mental illness is? Why do not you read up on some psychology.', 'this is not true. your sweeping stories you do know you a eye and say. for', "That is a sad tragedy. However,

## Final Table Outputs

In [20]:
test_y_text

In [21]:
#%% Bajar outputs
test_x_text = save_data('backup/test_x_text.pkl', None)
test_y_text = save_data('backup/test_y_text.pkl', None)

df_output_MAP = save_data('backup/df_output_MAP_EN.pkl', None)
df_output_SIM = save_data('backup/df_output_SIM_EN.pkl', None)

test_y_text_str = [y[0] for y in test_y_text]

df_output_EN = pd.DataFrame({'x_text': test_x_text, 'y_text': test_y_text_str,
                          'output_SIM': df_output_SIM['output'],
                          'output_MAP': df_output_MAP['output']})

df_output_EN = save_data('backup/df_output_EN.pkl', df_output_EN)

The file backup/test_x_text.pkl exist and will returned
The file backup/test_y_text.pkl exist and will returned
The file backup/df_output_MAP_EN.pkl exist and will returned
The file backup/df_output_SIM_EN.pkl exist and will returned
Data has been saved to backup/df_output_EN.pkl.


In [ ]:
df_output_EN

,x_text,y_text,output_SIM,output_MAP
0,"wow, what a rude cunt",If someone is rude it is better to ignore them...,"Wow, pipe down. Why so angry?","wow, tea."
1,white supremacist...anyone who is not a nigger...,There is a heavy amount of irony in what you a...,Most child abusers in the UK are white men so ...,Says who?
2,"Muh dick is the nigger sockpuppet line, ugly c...",Insults based on race are not ok.,"The world is not lost. We all face challenges,...","The world is not lost. We all face challenges,..."
3,Only pussies & jews like to silence. Why does ...,Insults based on sexual orientation or religio...,why you you think that what is just an entire ...,Why? What is wrong about learning about a worl...
4,A queer nigger on a sissy bike . The bike need...,Use of the slurs here is unacceptable in our d...,You cannot be racist towards an ideology. You ...,"Everyone is a potential everything, what would..."
...,...,...,...,...
2918,That's the last straw. I'm going to boycott n...,Hateful words and/or racial slurs are NOT tole...,Do you know that anti-Islamism is actually a f...,do you eat it my
2919,Only a Demorat cocksucking pussy hating faggot...,"We cannot support the use of such vulgar, raci...",this is bigotry.,why? this is quite hatred.
2920,snowflake cunt,Using words that profanely refer to parts of a...,this is a unusual lie. stain trouble.,source news
2921,The MSM will never report on themselves or hel...,"Hey there, derogatory comments are not product...",I agree that we need to tackle this serious br...,Then why did we ask them to come in the first ...


: 